# Bibliotheken

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os

# load Knowledge Base

In [2]:
loader3 = PyPDFLoader("../knowledgeBase/verbraucher_information.pdf")#path für Knowledge Base
documents_verbraucher_information = loader3.load_and_split()
print(f"SeiteAnzahl_documents_verbraucher_information: {len(documents_verbraucher_information)}")
knowledgeBase=documents_verbraucher_information
print(f"SeiteAnzahl_knowledgeBase: {len(knowledgeBase)}")
#print(documents[0].page_content)

SeiteAnzahl_documents_verbraucher_information: 114
SeiteAnzahl_knowledgeBase: 114


# chunking

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_1024 = text_splitter.split_documents(knowledgeBase)
print(len(texts_1024))

426


In [19]:
print(texts_1024[0].page_content)

200.0005.206 Stand 10.2023AUTO / KLASSIK-GARANT 2.0
ALLGEMEINE INFORMATIONEN
VERSICHERUNGSBEDINGUNGEN
MERKBLATT ZUR ANZEIGEPFLICHTVERLETZUNG
DATENSCHUTZHINWEISE
STAND 10/2023VERBRAUCHERINFORMATION 
NUMMER KN1023 PC


# embeddings

In [4]:
model_deutsche_telekom = HuggingFaceEmbeddings(model_name="deutsche-telekom/gbert-large-paraphrase-cosine")

c:\Users\ecoskun\Desktop\BachelorarbeitAbgabe\bachelorarbeit_eyuep_coskun_3594788\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Speicher Embeddings in db

In [5]:
db_name = "chroma_db_wizard"
db_path = os.path.join(db_name)
if os.path.exists(db_path):
    db=Chroma(persist_directory="chroma_db_wizard", embedding_function=model_deutsche_telekom)# wenn db schon erstellt wurde,load db
else:
    db = Chroma.from_documents(texts_1024, model_deutsche_telekom, persist_directory="chroma_db_wizard")# erstellt vektor datenbank

# Wizard-LLM

In [6]:
model_path = "C:/Users/ecoskun/AppData/Local/nomic.ai/GPT4All/wizardlm-13b-v1.2.Q4_0.gguf"#local path für LLM
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm= GPT4All(model=model_path, callbacks=callbacks, verbose=True,n_threads=16, temp=0.5)

# prompt template für RAG

In [7]:
from langchain.prompts import PromptTemplate
prompt_template='''
Du bist ein hilfreicher Assistent. Für die folgende Aufgabe stehen dir zwischen den tags BEGININPUT und ENDINPUT Kontext zur Verfügung. Die eigentliche Frage ist zwischen BEGININSTRUCTION und ENDINCSTRUCTION zu finden. Beantworten Sie die folgende Frage nur basierend auf dem angegebenen context auf Deutsch. Sollten diese keine Antwort enthalten, antworte, dass auf Basis der gegebenen Informationen keine Antwort möglich ist! 
USER: 
BEGININPUT{context}ENDINPUT
BEGININSTRUCTION {question} ENDINSTRUCTION 
ASSISTANT:
'''
PROMPT=PromptTemplate(
    template=prompt_template,input_variables=["context","question"]
)

# chain

In [8]:
chain_type_kwargs={"prompt":PROMPT}
qa_RAG_chain =RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs=chain_type_kwargs
)

# Pure-LLM

In [9]:
from langchain.chains import LLMChain
template = "Du bist ein hilfreicher Assistent. Antworte Frage auf Deutsch USER: {prompt} ASSISTANT:"
prompt = PromptTemplate.from_template(template)
callbacks = [StreamingStdOutCallbackHandler()]
# Verbose is required to pass to the callback manager
llm = GPT4All(model=model_path, callbacks=callbacks, verbose=True)
llm = GPT4All(model=model_path, backend="gptj", callbacks=callbacks, verbose=True, max_tokens=8092)
llm_chain = LLMChain(prompt=prompt, llm=llm)

## Load Test Daten

In [10]:
import json
try:
    with open('../test/testDaten/HannoverscheVerscicherung.json', 'r') as datei:
        json_data = json.load(datei)
except Exception as errror:
    print(f'errror: {errror}')

## Erstellt Json Ergebnis Datei

In [11]:
data = {
    "experiment_parameter": {
        "LLM_model": "GPT4All/wizardlm-13b-v1.2.Q4_0",
        "Embedding_model": "deutsche-telekom/gbert-large-paraphrase-cosine",
        "chunk_size": "1024",
        "chunk_overlap": 64,
        "LLM_temperatur": 0.5,
        "search_type": "similarity",
        "search_kwargs": "3",
        "chain_type": "stuff"
        },
        "wizard_ergebnisse": []
        }
with open("wizard_antworten2.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

# Speichert generierte Pure-LLM und RAG Antworten

In [11]:
def write_json(new_data, filename='wizard_antworten.json'):
    with open(filename,'r+') as file:
          # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["wizard_ergebnisse"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

# Generierung von Antworten

In [ ]:
import asyncio
import time
counter=1

result_data = {
    "ID":"",
    "frage": "",
    "ref_antwort": "",
    "RAG_kontex1":"",
    "RAG_kontex2":"",
    "RAG_kontex3":"",
    "RAG_wizard_antwort": "",
    "Pure_LLM_wizard_antwort":""
}
for question_answer in json_data:
    question = question_answer.get('frage', '')
    id=question_answer.get('ID','')
    print(f"{question}####################################################### {counter}")
    ref_ant = question_answer.get('ref_antwort', '')


    print(f"RAG Start Frage {counter}-------------------------------------------------------------------------")
    RAG_model_ant= qa_RAG_chain(question)# generiert RAG-Antwort auf Fragen
    print(f"RAG Fnish Frage {counter}-------------------------------------------------------------------------")

    
    print(f"Pure_LLM Start Frage {counter}-------------------------------------------------------------------------")
    pure_llm_antwort=llm_chain.run(question)# generiert Pure-LLM-Antwort auf Fragen
    print(f"Pure_LLM Fnish Frage {counter}-------------------------------------------------------------------------")
    
    kontext1=RAG_model_ant["source_documents"][0].page_content
    kontext2=RAG_model_ant["source_documents"][1].page_content
    kontext3=RAG_model_ant["source_documents"][2].page_content
    result_data["ID"]=id
    result_data['frage']=question
    result_data['ref_antwort']=ref_ant
    result_data['RAG_kontex1']=kontext1
    result_data['RAG_kontex2']=kontext2
    result_data['RAG_kontex3']=kontext3
    
    result_data['RAG_wizard_antwort']=RAG_model_ant["result"]
    result_data['Pure_LLM_wizard_antwort']=pure_llm_antwort
    write_json(result_data)# Funktion aufruf zur Speicherung der generierten RAG und Pure-LLM Antworten
    counter+=1